In [1]:
%pylab
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage
from nuclei_segmentation import otsu as ot
from nuclei_segmentation import dicescore as dsc
from nuclei_segmentation import preprocessing as pp
from nuclei_segmentation import local_thresholding as lt
from nuclei_segmentation import two_level_otsu as tlo

Using matplotlib backend: <object object at 0x000002ABF80FE520>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [18]:


def om(img,filtersize):
        img = lt.local_thresholding_mean_forward(img, 50, 200)
        return img

def oc(img,filtersize):
        img = lt.local_thresholding_counts(img, 50, 200, 0.5)
        return img

def m(img, filtersize):
        img=scipy.ndimage.median_filter(img, filtersize)
        return img

def g(img, filtersize):
        img=scipy.ndimage.gaussian_filter(img, filtersize)
        return img
        
def s(img,filtersize):
        img=pp.stretch(img)
        return img


    #functions = [om, oc, m, g, s]#
def process(image_list, function, filtersize):
        list = []
        
        for i in range(0,len(image_list)):
                list.append(function(image_list[i], filtersize))

        return list



def dscs(img_list, gt_list):
        list=[]
        for j in range(0,len(img_list)):
                list.append(dsc.dice(img_list[j],gt_list[j]))
        return list


In [19]:
def complete_analysis_local_otsu(dataset):
    if dataset == "N2DH-GOWT1":
        img = []
        path = r"data/Otsu_data/N2DH-GOWT1/img"
        for f in os.listdir(path):
            img.append(imread(os.path.join(path , f)))

        gt = []
        path = r"data/Otsu_data/N2DH-GOWT1/gt"
        for f in os.listdir(path):
            gt.append(pp.binarize(imread(os.path.join(path , f))))
        
        mfiltersize = 9
        gfiltersize=9
        
    if dataset == "N2DL_HeLa":

        img = []
        path = r"data/Otsu_data/N2DL-HeLa/img"
        for f in os.listdir(path):
            img.append(imread(os.path.join(path , f)))

        gt = []
        path = r"data/Otsu_data/N2DL-HeLa/gt"
        for f in os.listdir(path):
            gt.append(imread(os.path.join(path , f)))
        
        mfiltersize=2
        gfiltersize=4


    if dataset == "NIH3T3":
        img = []
        path = r"data/Otsu_data/NIH3T3/img"
        for f in os.listdir(path):
            img.append(imread(os.path.join(path , f)))

        gt = []
        path = r"data/Otsu_data/NIH3T3/gt"
        for f in os.listdir(path):
            gt.append(imread(os.path.join(path , f)))

        mfiltersize=16
        gfiltersize=8

    m_img = process(img,m, mfiltersize)
    print('m done')
    g_img = process(img,g,gfiltersize)
    print('g done')
    s_img = process(img,s,1)
    print('s done')
    sm_img = process(s_img,m,mfiltersize)
    print('sm done')
    sg_img = process(g_img,g,gfiltersize)
    print('sg done')
    om_img = process(img,om,1)
    print('om done')
    oc_img = process(img,oc,1)
    print('oc done')
    mom = process(m_img,om,1)
    print('mom done')
    moc = process(m_img,oc,1)
    print('moc done')
    gom = process(g_img,om,1)
    print('gom done')
    goc = process(g_img,oc,1)
    print('goc done')
    som = process(s_img, om, 1)
    print('som done')
    soc = process(s_img, oc, 1)
    print('soc done')
    smom = process(sm_img,om,1)
    print('smom done')
    smoc = process(sm_img,oc,1)
    print('smoc done')
    sgom = process(sg_img,om,1)
    print('sgom done')
    sgoc = process(sg_img,oc,1)
    print('sgoc done')

    #dice score for o, mo,go,so,smo,sgo:
    data=[dscs(om_img,gt),dscs(oc_img,gt),dscs(mom, gt), dscs(moc, gt), dscs(gom, gt), dscs(goc, gt), dscs(som, gt), dscs(soc,gt), dscs(smom,gt), dscs(smoc,gt), dscs(gmom,gt), dscs(gmoc,gt)]
    print('dscs done')
    boxplot_heading = 'Preprocessing methods - ' + dataset

    boxplot = pp.dataset_boxplot_global_otsu(data , boxplot_heading)

    
    


    
    

    






In [15]:
complete_analysis_local_otsu("NIH3T3")

m done
g done
s done
sm done
sg done


KeyboardInterrupt: 

In [ ]:
complete_analysis_local_otsu("N2DL_HeLa")